In [10]:
import io
import pandas as pd
import numpy as np

pd.options.display.max_colwidth = 20
pd.options.display.max_rows = 4

In [2]:
with open('./data/processed/dataset.pkl', 'rb') as f:
    DATASET_BINARY = f.read()

In [3]:
# file_path = './data/processed/dataset.pkl'
# df = pd.read_pickle(file_path, compression='zstd')
df_dataset = pd.read_pickle(io.BytesIO(DATASET_BINARY), compression='zstd')
df_dataset

,CD_CVM,CNPJ_CIA,DENOM_CIA,fs_type,is_cons,DT_REFER,VERSAO,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,94,92.693.019/0001-89,PANATLANTICA S.A.,1,True,2010-12-31,2,NaT,2009-12-31,-1,1,Ativo Total,True,NaN,211293000.0
1,94,92.693.019/0001-89,PANATLANTICA S.A.,1,False,2010-12-31,2,NaT,2009-12-31,-1,1,Ativo Total,True,NaN,191541000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17674197,518700,70.216.429/0001-04,BRAMEX - BRASIL ...,7,False,2013-12-31,1,2013-01-01,2013-12-31,0,7.08.04.03,Lucros Retidos /...,True,NaN,-1200611.0
17674198,518700,70.216.429/0001-04,BRAMEX - BRASIL ...,7,False,2013-12-31,1,2013-01-01,2013-12-31,0,7.08.05,Outros,True,NaN,486752.0


In [4]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17674199 entries, 0 to 17674198
Data columns (total 15 columns):
 #   Column         Dtype   
---  ------         -----   
 0   CD_CVM         category
 1   CNPJ_CIA       category
 2   DENOM_CIA      category
 3   fs_type        category
 4   is_cons        category
 5   DT_REFER       category
 6   VERSAO         category
 7   DT_INI_EXERC   category
 8   DT_FIM_EXERC   category
 9   ORDEM_EXERC    category
 10  CD_CONTA       category
 11  DS_CONTA       category
 12  ST_CONTA_FIXA  category
 13  COLUNA_DF      category
 14  VL_CONTA       float64 
dtypes: category(14), float64(1)
memory usage: 508.6 MB


In [5]:
df_dataset.query("CD_CVM == 9512")

,CD_CVM,CNPJ_CIA,DENOM_CIA,fs_type,is_cons,DT_REFER,VERSAO,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
3144784,9512,33.000.167/0001-01,PETROLEO BRASILE...,1,True,2010-12-31,3,NaT,2009-12-31,-1,1,Ativo Total,True,NaN,3.504189e+11
3144785,9512,33.000.167/0001-01,PETROLEO BRASILE...,1,False,2010-12-31,3,NaT,2009-12-31,-1,1,Ativo Total,True,NaN,3.189971e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3181896,9512,33.000.167/0001-01,PETROLEO BRASILE...,7,True,2021-09-30,1,2021-01-01,2021-09-30,0,7.08.05.01,Valor adicionado...,False,NaN,0.000000e+00
3181897,9512,33.000.167/0001-01,PETROLEO BRASILE...,7,False,2021-09-30,1,2021-01-01,2021-09-30,0,7.08.05.01,Valor adicionado...,False,NaN,0.000000e+00


In [6]:
class Company():
    def __init__(self, cvm_number: int):
        self.cvm_number = int(cvm_number)
    def assets(self):
        df = pd.read_pickle(io.BytesIO(DATASET_BINARY), compression='zstd')
        df.query("CD_CVM == @self.cvm_number", inplace=True)
        df.drop(columns=['CNPJ_CIA', 'DENOM_CIA', 'VERSAO'], inplace=True)
        # df = df_dataset.query("CD_CVM == @self.cvm_number")
        df.sort_values(['DT_REFER', 'ORDEM_EXERC'], inplace=True)
        df.reset_index(drop=True, inplace=True)
        return df

In [29]:
petro = Company(9512)
df = petro.assets()
df

,CD_CVM,fs_type,is_cons,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,9512,1,True,2010-12-31,NaT,2009-12-31,-1,1,Ativo Total,True,NaN,3.504189e+11
1,9512,1,False,2010-12-31,NaT,2009-12-31,-1,1,Ativo Total,True,NaN,3.189971e+11
...,...,...,...,...,...,...,...,...,...,...,...,...
37112,9512,7,True,2021-09-30,2021-01-01,2021-09-30,0,7.08.05.01,Valor adicionado...,False,NaN,0.000000e+00
37113,9512,7,False,2021-09-30,2021-01-01,2021-09-30,0,7.08.05.01,Valor adicionado...,False,NaN,0.000000e+00


In [17]:
df.COLUNA_DF.unique()

[NaN, 'Capital Social ..., 'Reservas de Cap..., 'Reservas de Lucro', 'Lucros ou Preju..., 'Outros Resultad..., 'Patrimônio Líqu..., 'Participação do..., 'Patrimônio Líqu...]
Categories (8, object): ['Capital Social ..., 'Lucros ou Preju..., 'Outros Resultad..., 'Participação do..., 'Patrimônio Líqu..., 'Patrimônio Líqu..., 'Reservas de Cap..., 'Reservas de Lucro']

In [30]:
df = df.astype({
    'CD_CVM': 'object',
    'fs_type': np.int8,
    'is_cons': bool,
    'DT_REFER': 'datetime64',
    'DT_INI_EXERC': 'datetime64',
    'DT_FIM_EXERC': 'datetime64',
    'ORDEM_EXERC': np.int8,
    'CD_CONTA': 'object',
    'DS_CONTA': 'object',
    'ST_CONTA_FIXA': bool,
    'COLUNA_DF': 'object',
    'VL_CONTA': float
})
df

,CD_CVM,fs_type,is_cons,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,9512,1,True,2010-12-31,NaT,2009-12-31,-1,1,Ativo Total,True,NaN,3.504189e+11
1,9512,1,False,2010-12-31,NaT,2009-12-31,-1,1,Ativo Total,True,NaN,3.189971e+11
...,...,...,...,...,...,...,...,...,...,...,...,...
37112,9512,7,True,2021-09-30,2021-01-01,2021-09-30,0,7.08.05.01,Valor adicionado...,False,NaN,0.000000e+00
37113,9512,7,False,2021-09-30,2021-01-01,2021-09-30,0,7.08.05.01,Valor adicionado...,False,NaN,0.000000e+00


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37114 entries, 0 to 37113
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   CD_CVM         37114 non-null  object        
 1   fs_type        37114 non-null  int8          
 2   is_cons        37114 non-null  bool          
 3   DT_REFER       37114 non-null  datetime64[ns]
 4   DT_INI_EXERC   25026 non-null  datetime64[ns]
 5   DT_FIM_EXERC   37114 non-null  datetime64[ns]
 6   ORDEM_EXERC    37114 non-null  int8          
 7   CD_CONTA       37114 non-null  object        
 8   DS_CONTA       37114 non-null  object        
 9   ST_CONTA_FIXA  37114 non-null  bool          
 10  COLUNA_DF      13916 non-null  object        
 11  VL_CONTA       37114 non-null  float64       
dtypes: bool(2), datetime64[ns](3), float64(1), int8(2), object(4)
memory usage: 2.4+ MB


In [32]:
df.DT_REFER.min()

Timestamp('2010-12-31 00:00:00')